# SageMaker Serverless Inference 

Amazon SageMaker Serverless Inference is a purpose-built inference option that makes it easy to deploy and scale ML models. 

Serverless Inference is ideal for workloads which have **idle periods** between traffic peaks and can tolerate **cold starts**. Serverless endpoints also automatically launch compute resources and scale them in and out depending on traffic, eliminating the need to choose instance types or manage scaling policies. Tiis is ideal for scenarios that we have constrains in cost and at the same time we have no idea about the type of instance we need.

For this notebook we'll be working with the **SageMaker XGBoost Algorithm** to train a model and then deploy a serverless endpoint. We will be using the public S3 Abalone regression dataset for this example.

In [1]:
! pip install sagemaker botocore boto3 awscli --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 916.9/916.9 kB 36.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for botocore from https://files.pythonhosted.org/packages/64/93/d472917cc8000157d1220f18edf175e06b7f1ac92970e7cdb719d4405b5a/botocore-1.31.78-py3-none-any.whl.metadata
  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/17/a4/e161e777d445a5b1029eff4c4272896410d7dac55e17f2889bd9a3517eed/boto3-1.28.78-py3-none-any.whl.metadata
  Obtaining dependency information for awscli from https://files.pythonhosted.org/packages/4a/3a/2bc87e42c30f79423c8ded2efd9e94a0d50cba07e109d08ceb76861272d2/awscli-1.29.78-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 72.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 88.8 MB/s eta 0:00:00:00:01
  Created wheel for sagema

In [2]:
# Setup clients
import boto3

client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")

To begin, we import the AWS SDK for Python (Boto3) and set up our environment, including an IAM role and an S3 bucket to store our data.

In [3]:
import boto3
import sagemaker
from sagemaker.estimator import Estimator

boto_session = boto3.session.Session()
region = boto_session.region_name
print(region)

sagemaker_session = sagemaker.Session()
base_job_prefix = "xgboost-example"
role = sagemaker.get_execution_role()
print(role)

default_bucket = 'day07-mk1'
s3_prefix = base_job_prefix

training_instance_type = "ml.m5.xlarge"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
us-east-1
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
arn:aws:iam::239630988601:role/fast-ai-academic-1-Student-Azure


Retrieve the Abalone dataset from a publicly hosted S3 bucket.

In [4]:
# retrieve data
! curl https://sagemaker-sample-files.s3.amazonaws.com/datasets/tabular/uci_abalone/train_csv/abalone_dataset1_train.csv > abalone_dataset1_train.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  131k  100  131k    0     0  1004k      0 --:--:-- --:--:-- --:--:-- 1008k


Upload the Abalone dataset to the default S3 bucket.

In [5]:
# upload data to S3
!aws s3 cp abalone_dataset1_train.csv s3://{default_bucket}/xgboost-regression/train.csv

upload: ./abalone_dataset1_train.csv to s3://day07-mk1/xgboost-regression/train.csv


### Model Training

Now, we train an ML model using the XGBoost Algorithm. In this example, we use a SageMaker-provided XGBoost container image and configure an estimator to train our model.

In [6]:
from sagemaker.inputs import TrainingInput

training_path = f"s3://{default_bucket}/xgboost-regression/train.csv"
train_input = TrainingInput(training_path, content_type="text/csv")

In [7]:
model_path = f"s3://{default_bucket}/{s3_prefix}/xgb_model"

# retrieve xgboost image
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
    instance_type=training_instance_type,
)

# Configure Training Estimator
xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    output_path=model_path,
    sagemaker_session=sagemaker_session,
    role=role,
)

# Set Hyperparameters
xgb_train.set_hyperparameters(
    objective="reg:linear",
    num_round=50,
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.7,
    silent=0,
)

Train the model on the Abalone dataset.

In [8]:
# Fit model
xgb_train.fit({"train": train_input})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-11-05-16-42-21-650


2023-11-05 16:42:21 Starting - Starting the training job...
2023-11-05 16:42:37 Starting - Preparing the instances for training......
2023-11-05 16:43:32 Downloading - Downloading input data...
2023-11-05 16:44:08 Training - Downloading the training image..[2023-11-05 16:44:34.410 ip-10-0-118-156.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:linear to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single node training.
[16:44:34] 2923x8 matrix with 23384 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2023-11-05 16:44:3

### Deployment

After training the model, retrieve the model artifacts so that we can deploy the model to an endpoint.

In [9]:
# Retrieve model data from training job
model_artifacts = xgb_train.model_data
model_artifacts

's3://day07-mk1/xgboost-example/xgb_model/sagemaker-xgboost-2023-11-05-16-42-21-650/output/model.tar.gz'

### Model Creation

Create a model by providing your model artifacts, the container image URI, environment variables for the container (if applicable), a model name, and the SageMaker IAM role.

In [10]:
from time import gmtime, strftime

model_name = "xgboost-serverless" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Model name: " + model_name)

# dummy environment variables
byo_container_env_vars = {"SAGEMAKER_CONTAINER_LOG_LEVEL": "20", "SOME_ENV_VAR": "myEnvVar"}

create_model_response = client.create_model(
    ModelName=model_name,
    Containers=[
        {
            "Image": image_uri,
            "Mode": "SingleModel",
            "ModelDataUrl": model_artifacts,
            "Environment": byo_container_env_vars,
        }
    ],
    ExecutionRoleArn=role,
)

print("Model Arn: " + create_model_response["ModelArn"])

Model name: xgboost-serverless2023-11-05-16-46-38
Model Arn: arn:aws:sagemaker:us-east-1:239630988601:model/xgboost-serverless2023-11-05-16-46-38


### Endpoint Configuration Creation

This is where you can adjust the **Serverless Configuration** for your endpoint. The current max concurrent invocations for a single endpoint, known as MaxConcurrency, can be any value from **1 to 200**, and MemorySize can be any of the following: **1024 MB, 2048 MB, 3072 MB, 4096 MB, 5120 MB, or 6144 MB.**

In [13]:
xgboost_epc_name = "xgboost-serverless-epc" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=xgboost_epc_name,
    ProductionVariants=[
        {
            "VariantName": "byoVariant",
            "ModelName": model_name,
            "ServerlessConfig": {
                "MemorySizeInMB": 3072,
                "MaxConcurrency": 1,
            },
        },
    ],
)

print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

Endpoint Configuration Arn: arn:aws:sagemaker:us-east-1:239630988601:endpoint-config/xgboost-serverless-epc2023-11-05-16-47-46


### Serverless Endpoint Creation

Now that we have an endpoint configuration, we can create a **serverless endpoint** and deploy our model to it. When creating the endpoint, provide the name of your endpoint configuration and a name for the new endpoint.

In [14]:
endpoint_name = "xgboost-serverless-ep" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=xgboost_epc_name,
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

Endpoint Arn: arn:aws:sagemaker:us-east-1:239630988601:endpoint/xgboost-serverless-ep2023-11-05-16-47-47


Wait until the endpoint status is InService before invoking the endpoint.

In [15]:
# wait for endpoint to reach a terminal state (InService) using describe endpoint
import time

describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)

while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)

describe_endpoint_response

Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService


{'EndpointName': 'xgboost-serverless-ep2023-11-05-16-47-47',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:239630988601:endpoint/xgboost-serverless-ep2023-11-05-16-47-47',
 'EndpointConfigName': 'xgboost-serverless-epc2023-11-05-16-47-46',
 'ProductionVariants': [{'VariantName': 'byoVariant',
   'DeployedImages': [{'SpecifiedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3',
     'ResolvedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost@sha256:1b38de0dbca9bfe90990430d0c4639fe5660c5dbc3964e8af33621c3b07a5a97',
     'ResolutionTime': datetime.datetime(2023, 11, 5, 16, 47, 49, 242000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 0,
   'CurrentServerlessConfig': {'MemorySizeInMB': 3072, 'MaxConcurrency': 1}}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2023, 11, 5, 16, 47, 48, 433000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2023, 11, 5, 1

### Endpoint Invocation

Invoke the endpoint by sending a request to it. The following is a sample data point grabbed from the CSV file downloaded from the public Abalone dataset.

In [16]:
%%time
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=b".345,0.224414,.131102,0.042329,.279923,-0.110329,-0.099358,0.0",
    ContentType="text/csv",
)

print(response["Body"].read())

b'4.566554546356201'
CPU times: user 26.2 ms, sys: 0 ns, total: 26.2 ms
Wall time: 179 ms


Let's review the above output : CPU times: user 36.8 ms, sys: 0 ns, total: 36.8 ms 

user 36.8 ms ---> that shows the amount of time spent executing user-level code, which is code that is written in a higher-level language (in this case Python).

sys 0 ns --> that shows the amount of time spent executing system-level code, which is code that interacts directly with the operating system. In this case is very small value that is considered as 0.

total 36.8 --> The total amount of CPU time spent executing the code (i.e., the sum of user and sys values).

Wall time: 326 ms --> the wall-clock time is the elapsed time between the start and end of the code block, as measured by a clock on the wall (i.e., a real-world clock). This includes time spent waiting for input/output operations, sleep calls, or other types ofactions or blocking calls that may not consume any CPU time but still takes time.

In this example, the wall time was 326 ms, which is significantly longer than the CPU time of 36.8 ms. This indicates that the code spent a significant amount of time waiting for other things that as you are aware thar is becuase no container was there before the first request come.

## Assignment:

1- Now that you learn how to get the time spent to get the reponse, go back to that real-time notebook and compare the CPU time and Wall time of real time with serverless and explain why they are different.

2- Use the above cell to re-send an inferencing request to the serverless endpoint immidiately after the first request. Do you see an improvmen in the wall time. If yes why?

## Please delete the endpoint in the console before you close this session